In [ ]:
import cv2
import numpy as np
import face_recognition
import mysql.connector
import dlib
from flask import Flask, request, jsonify, redirect, url_for

app = Flask(__name__)
# Database Configuration
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "police_case_management"
}


def connect_db():
    """Ensures database connection is always valid."""
    try:
        conn = mysql.connector.connect(**db_config)
        return conn
    except mysql.connector.Error as err:
        print(f"Database Error: {err}")
        return None


@app.route('/register_face', methods=['POST'])
def register_face():
    """Stores user's face encoding in the database."""
    username = request.form['username']
    role = request.form['role']

    # Capture frame from webcam
    video_capture = cv2.VideoCapture(0)
    ret, frame = video_capture.read()
    video_capture.release()

    if not ret:
        return jsonify({"error": "Failed to capture face image"}), 500

    # Encode the face
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    if not face_encodings:
        return jsonify({"error": "No face detected"}), 400

    # Convert the encoding to a format that can be stored in MySQL
    encoding_str = ",".join(map(str, face_encodings[0]))

    conn = connect_db()
    if conn is None:
        return jsonify({"error": "Database connection failed"}), 500

    cursor = conn.cursor()
    query = "INSERT INTO users (username, role, face_encoding) VALUES (%s, %s, %s)"
    cursor.execute(query, (username, role, encoding_str))
    conn.commit()
    conn.close()

    return jsonify({"success": "User registered successfully"}), 200


@app.route('/login_face', methods=['POST'])
def login_face():
    """Validates user by comparing stored face encoding."""
    video_capture = cv2.VideoCapture(0)
    ret, frame = video_capture.read()
    video_capture.release()

    if not ret:
        return jsonify({"error": "Failed to capture face image"}), 500

    # Encode the face
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    if not face_encodings:
        return jsonify({"error": "No face detected"}), 400

    conn = connect_db()
    if conn is None:
        return jsonify({"error": "Database connection failed"}), 500

    cursor = conn.cursor()
    cursor.execute("SELECT id, username, role, face_encoding FROM users")
    users = cursor.fetchall()
    conn.close()

    for user in users:
        try:
            user_encoding = np.array([float(x) for x in user[3].split(',')])  # Convert stored string to numpy array
            match = face_recognition.compare_faces([user_encoding], face_encodings[0])[0]

            if match:
                return jsonify({
                    "success": True,
                    "username": user[1],
                    "role": user[2],
                    "redirect": url_for('dashboard', view='cards') if user[2] == "admin" else url_for('dashboard', view='officer')
                }), 200
        except ValueError:
            continue  # Skip any invalid stored encodings

    return jsonify({"error": "Face not recognized"}), 403


if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1